This IPYNB file was generated automatically by MOA GUI.<br>
Task name: moa.tasks.EvaluatePrequentialDelayed<br>
## Libraries importing


In [1]:
%maven nz.ac.waikato.cms.moa:moa:2019.05.0
%classpath "moa-flow-core.jar"
import moa.classifiers.meta.AdaptiveRandomForest;
import moa.streams.generators.RandomTreeGenerator;
import moa.evaluation.WindowClassificationPerformanceEvaluator;
import moa.evaluation.LearningEvaluation;
import moa.evaluation.preview.LearningCurve;
import moa.core.TimingUtils;
import moa.tasks.MainTask;
import com.yahoo.labs.samoa.instances.Instance;
import moa.core.Example;
import moa.core.Measurement;
import java.util.LinkedList;
import moaflow.sink.DrawTable;
import moa.core.Utils;
import moa.core.InstanceExample;


## Configuring learner, stream and evaluator


In [2]:
String learnerString = "meta.AdaptiveRandomForest -l (ARFHoeffdingTree -e 2000000 -g 75 -s GiniSplitCriterion -c 0.01 -l MC) -o (Percentage (M * (m / 100))) -m 80";
AdaptiveRandomForest adap = new AdaptiveRandomForest();adap.getOptions().setViaCLIString("-l (ARFHoeffdingTree -e 2000000 -g 75 -s GiniSplitCriterion -c 0.01 -l MC) -o (Percentage (M * (m / 100))) -m 80");
adap.prepareForUse();

String streamString = "generators.RandomTreeGenerator";
RandomTreeGenerator rand = new RandomTreeGenerator();
rand.prepareForUse();
adap.setModelContext(rand.getHeader());

String evaluatorString = "WindowClassificationPerformanceEvaluator";
WindowClassificationPerformanceEvaluator wind = new WindowClassificationPerformanceEvaluator();


## Setting environmental parameters


In [3]:
int maxInstances = 100000;
long instancesProcessed = 0;
int maxSeconds = -1;
int secondsElapsed = 0;
LearningCurve learningCurve = new LearningCurve("learning evaluation instances");
File dumpFile = new File("null");
PrintStream immediateResultStream = null;
if (dumpFile != null) {
    try {
        if (dumpFile.exists()) {
            immediateResultStream = new PrintStream(
                    new FileOutputStream(dumpFile, true), true);
        } else {
            immediateResultStream = new PrintStream(
                    new FileOutputStream(dumpFile), true);
        }
    } catch (Exception ex) {
        throw new RuntimeException(
                "Unable to open immediate result file: "  + dumpFile, ex);
    }
}
File outputPredictionFile = null;
PrintStream outputPredictionResultStream = null;
if (outputPredictionFile != null) {
    try {
        if (outputPredictionFile.exists()) {
            outputPredictionResultStream = new PrintStream(
                    new FileOutputStream(outputPredictionFile, true), true);
        } else {
            outputPredictionResultStream = new PrintStream(
                    new FileOutputStream(outputPredictionFile), true);
        }
    } catch (Exception ex) {
        throw new RuntimeException(
                 "Unable to open prediction result file: " + outputPredictionFile, ex);
    }
}
LinkedList<Example> trainInstances = new LinkedList<Example>();
boolean firstDump = true;
boolean firstPrint = true;
boolean preciseCPUTiming = TimingUtils.enablePreciseTiming();
long evaluateStartTime = TimingUtils.getNanoCPUTimeOfCurrentThread();
long lastEvaluateStartTime = evaluateStartTime;
double RAMHours = 0.0;


## Testing & training, exporting result


In [4]:
DrawTable table = new DrawTable();
System.out.println( "Evaluating learner...");
while (rand.hasMoreInstances()
        && ((maxInstances < 0) || (instancesProcessed < maxInstances))
        && ((maxSeconds < 0) || (secondsElapsed < maxSeconds))) {
    instancesProcessed++;
    Example currentInst = rand.nextInstance();
    boolean trainOnInitialWindow = false;
    boolean trainInBatches = false;
    int initialWindowSize = 1000;
    int delayLength = 1000;
    if (instancesProcessed <= initialWindowSize) {
        if (trainOnInitialWindow) {
            adap.trainOnInstance(currentInst);
        } else if ((initialWindowSize - instancesProcessed) < delayLength) {
            trainInstances.addLast(currentInst);
        }
    } else {
        trainInstances.addLast(currentInst);
        if (delayLength < trainInstances.size()) {
            if (trainInBatches) {
                // Do not train on the latest instance, otherwise
                // it would train on k+1 instances
                while (trainInstances.size() > 1) {
                    Example trainInst = trainInstances.removeFirst();
                    adap.trainOnInstance(trainInst);
                }
            } else {
                Example trainInst = trainInstances.removeFirst();
                adap.trainOnInstance(trainInst);
            }
        }
        // Remove class label from test instances.
        Instance testInstance = ((Instance) currentInst.getData()).copy();
        testInstance.setMissing(testInstance.classAttribute());
        testInstance.setClassValue(0.0);
        Example testInst = new InstanceExample(testInstance);
        double[] prediction = adap.getVotesForInstance(testInst);
        testInstance = ((Instance) currentInst.getData()).copy();
        testInst = new InstanceExample(testInstance);
        // Output prediction
        if (outputPredictionFile != null) {
            int trueClass = (int) ((Instance) currentInst.getData()).classValue();
            outputPredictionResultStream.println(Utils.maxIndex(prediction) + ","  + ( 
                    ((Instance) testInst.getData()).classIsMissing() == true ? " ? " : trueClass));
        }
        wind.addResult(testInst, prediction);
    if (instancesProcessed % 10000 == 0
            ||  rand.hasMoreInstances() == false) {
        long evaluateTime = TimingUtils.getNanoCPUTimeOfCurrentThread();
        double time = TimingUtils.nanoTimeToSeconds(evaluateTime - evaluateStartTime);
        double timeIncrement = TimingUtils.nanoTimeToSeconds(evaluateTime - lastEvaluateStartTime);
        double RAMHoursIncrement = adap.measureByteSize() / (1024.0 * 1024.0 * 1024.0); //GBs
        RAMHoursIncrement *= (timeIncrement / 3600.0); //Hours
        RAMHours += RAMHoursIncrement;
        lastEvaluateStartTime = evaluateTime;
        learningCurve.insertEntry(new LearningEvaluation(
                new Measurement[]{
                        new Measurement(
                                "learning evaluation instances",
                                instancesProcessed),
                        new Measurement(
                                "evaluation time ("
                                        + (preciseCPUTiming ? "cpu "
                                        : "") + "seconds)",
                                time),
                        new Measurement(
                                "model cost (RAM-Hours)",
                                RAMHours)
                },
                wind, adap));
        if (immediateResultStream != null) {
            if (firstDump) {
                immediateResultStream.println(learningCurve.headerToString());
                firstDump = false;
            }
            immediateResultStream.print(learnerString + "," + streamString);
            immediateResultStream.println(learningCurve.entryToString(learningCurve.numEntries() - 1));
            immediateResultStream.flush();
        }
        if (firstPrint) {
           System.out.println(learnerString + "," + streamString);
           System.out.print(table.headerLine(learningCurve.headerToString()));
           firstPrint = false;
        }
        System.out.print(table.bodyLine(learningCurve.entryToString(learningCurve.numEntries() - 1)));
        System.out.flush();
    }
}
}
double time = TimingUtils.nanoTimeToSeconds(TimingUtils.getNanoCPUTimeOfCurrentThread()- evaluateStartTime);
System.out.println(instancesProcessed + " instances processed in " + time + " seconds.");
if (immediateResultStream != null) {
    immediateResultStream.close();
}
